# Iteration 

## The Enhanced For Loop

Java allows us to iterate through Lists and Sets using a convenient shorthand syntax sometimes called the "foreach" or "enhanced for" loop.

In [ ]:
Set<Integer> javaset = new HashSet<>();
javaset.add(5);
javaset.add(23);
javaset.add(42);
for (int i: javaset) {
    System.out.println(i); 
}

However, turns out we can't use the enhanced for loop with the `ArraySet`! 

In [ ]:
ArraySet<Integer> aset = new HashSet<>();
javaset.add(5);
javaset.add(23);
javaset.add(42);
for (int i: aset) {
    System.out.println(i); 
}

// =======OUTPUT =======
$ error: for-each not applicable to expression type
            for (int i: aset) {
                
    required: array or java.lang.Iterable
    found:    ArraySet<Integer> 

## How Iteration Really Works

Turns out the enhanced for loop approach, which we'll call the "nice iteration", can be replicated using the following "ugly" iteration,

In [ ]:
// Nice Iteration
Set<Integer> javaset = new HashSet<Integer>();
javaset.add(5);
javaset.add(23);
javaset.add(42);
for (int x : javaset) {
    System.out.println(x);
}

In [ ]:
// Ugly Iteration
import java.util.Iterator; // Import this outside the class definition

Set<Integer> javaset = new HashSet<Integer>();
javaset.add(5);
javaset.add(23);
javaset.add(42);
Iterator<Integer> seer = javaset.iterator();

while (seer.hasNext()) { // as long as seer still has elements available
    System.out.println(seer.next()); 
}

Notice in ugly iteration, we instantiate an `Iterator` object whose methods we'll use (e.g. `hasNext()`) 

We can think of the `seer` as the pointer. Initially, `seer` points at nothing.

1. `hasNext()` returns True because the next value is 5
    * `seer.next()` then returns 5
2. `hasNext()` returns True because the next value is 23
    * `seer.next()` then returns 23
3. `hasNext()` returns True because the next value is 42
    * `seer.next()` then returns 42
4. `hasNExt()` then returns False because there's no other element after 42

In [ ]:
1. `hasNext()

## The Secret of the Enhanced For Loop

![](images/secret.png)

**The code on the left is just a shorthand for the code on the right.**

For the code on the right to compile, what does the compiler need to check?

1. Does the `Set` interface have an `iterator()` method?
2. Does the `Set` interface have `next/hasNext()` method?
3. Does the `Iterator` interface have an `iterator` method?
4. Does the `Iterator` interface have `next()` and `hasNext()` methods?

**Ans**: 1 and 4

## Supporting Ugly Iteration in ArraySets

To support ugly iteration:

1. Add an `iterator()` method to `ArraySet()`
    * This `iterator()` method should return an `Iterator<T>`
2. The `Iterator<T>` that we return should have a useful `hasNext()` and `next()` method

In [ ]:
public interface Iterator<T> {
    boolean hasNext();
    T next();
}

There are a few steps to do this:

#### `Iterator<T> iterator()` method

First, create a method that returns an iterator.

In [ ]:
public Iterator<T> iterator(){
    return...??
}

What do we return? We haven't made a class for this, so let's implement one! 

In [ ]:
private class ArraySetIterator implements Iterator<T>{
    public boolean hasNext(){
        ...
    }
    
    public Integer next(){
        ...
    }
}

Now we can go back to the `iterator()` method,

In [ ]:
public Iterator<T> iterator() {
    return new ArraySetIterator();
}

Now the `iterator()` method is complete, but we still need to complete the `ArraySetIterator` class. How do we complete it?

We need to emulate the `seer` pointer functionality: a pointer that keeps track of the position along the array.

In [ ]:
private class ArraySetIterator implements Iterator<T> {
    private int pointer;
    public ArraySetIterator() {
        pointer = 0;
    }
}

Now that we have a pointer, we can complete the implementation for `hasNext()` and `next()` method.

In [ ]:
public boolean hasNext(){
    // As long as the pointer is less than the size of the array,
    // the pointer still haven't reached the end of the array
    return pointer < size
}

In [ ]:
public T next() {
    T returnItem = items[pointer]; // Assign the item that's going to be returned to 'returnItem'
    pointer += 1;
    return returnItem;
}

We just have finished implementing an iterator object that allows us to walk through our array! 

In [ ]:
private class ArraySetIterator implements Iterator<T> {
    private int pointer;
    public ArraySetIterator() {
        pointer = 0;
    }
    
    public boolean hasNext(){
        // As long as the pointer is less than the size of the array,
        // the pointer still haven't reached the end of the array
        return pointer < size
    }
    
    public T next() {
        T returnItem = items[pointer]; // Assign the item that's going to be returned to 'returnItem'
        pointer += 1;
        return returnItem;
    }
}

public Iterator<T> iterator() {
    return new ArraySetIterator();
}

## The Enhanced For Loop

Our code now supports "ugly" iteration, but enhanced for loop still doesn't work.

In [ ]:
ArraySet<Integer> aset = new ArraySet<>();
aset.add(5);
aset.add(23);
aset.add(42);
for (int i: aset){
    ...
}

// ==== Output ====
error: for-each not applicable to expression type
        for (int i: aset) {
            
    required: array or java.lang.iterable
    found:    ArraySet<Integer>

The problem: Java isn't smart enough to realize that our `ArraySet` has an `iterator()` method
* Luckily there's an interface for this: `Iterable<T>`

## For-each Iteration and ArraySets

To support enhanced for loop, we need to make `ArraySet` class implement the `Iterable` interface.
* There are also some default methods in `Iterable`, not shown

![](images/iterable.png)

In [ ]:
import ...

public class ArraySet<T> implements Iterable<T> {
    private T[] items;
}

This way, we're declaring that ArraySet has an `iterator()` method. Without `implements Iterable<T>`, even though our code supports ugly iteration, Java doesn't know that now `ArraySet` has a usable `iterator()`.

## The Iterable Interface

This is how `Set` works as well.

![](images/set.png)

In [ ]:
public interface Iterable<T> {
    Iterator<T> iterator();
    ...
}

In [ ]:
public class Collection<E> extends Iterable<E> {
    public Iterator<E> iterator();
}

In [ ]:
public class Set<E> extends Collection<E> {
    public Iterator<E> iterator();
}

This means we can use enhanced for loop for sets because Java knows sets inherit the `iterator()` method.

## Iteration Summary

To support the enhanced for loop (in other words, the ability to do the following),

In [ ]:
for (int i: aset){
    System.out.println(i);
}

* Add an `iterator()` method that returns an `Iterator<T>` to the class
* Make sure the `Iterator<T>` object returned has a useful `hasNext()` and `next()` method
* Add `implements Iterable<T>` to the line defining the class in the first place